In [1]:
import requests
import warnings
from tqdm import tqdm, trange
import pandas as pd 
import streamlit as st 
import yfinance as yf
from datetime import datetime
from dateutil.relativedelta import relativedelta
import plotly_express  as px
import plotly.graph_objects as go
import random
import pickle
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [28]:
def get_stock_data(tickerI):
    tickerL = tickerI
    
    
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{tickerL}?period=annual&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    annual_is_df = pd.DataFrame(response.json())
    if annual_is_df.shape == (0, 0):
        annual_is_df =  pd.DataFrame((float('Nan'), float('Nan')))
        last_date_annual = float('Nan')
    else:
        last_date_annual = annual_is_df.loc[0,'date']
        annual_is_df = annual_is_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        annual_is_df = annual_is_df.add_suffix(f'(FY)')
        
    
    
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{tickerL}?period=quarterly&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    quarterly_is_df = pd.DataFrame(response.json())
    if quarterly_is_df.shape == (0, 0):
        quarterly_is_df =  pd.DataFrame((float('Nan'), float('Nan')))
        last_date_quarterly = float('Nan')
    else:
        last_date_quarterly = quarterly_is_df.loc[0,'date']
        quarterly_is_df = quarterly_is_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        quarterly_is_df = quarterly_is_df.add_suffix(f'(FQ)')
        
    
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{tickerL}?period=annual&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    annual_fp_df = pd.DataFrame(response.json())
    if annual_fp_df.shape == (0, 0):
        annual_fp_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        annual_fp_df = annual_fp_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        annual_fp_df = annual_fp_df.add_suffix(f'(FY)')
    
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{tickerL}?period=quarterly&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    quarterly_fp_df = pd.DataFrame(response.json())
    if quarterly_fp_df.shape == (0, 0):
        quarterly_fp_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        quarterly_fp_df = quarterly_fp_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        quarterly_fp_df = quarterly_fp_df.add_suffix(f'(FQ)')
    
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{tickerL}?period=annual&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    annual_cf_df = pd.DataFrame(response.json())
    if annual_cf_df.shape == (0, 0):
        annual_cf_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        annual_cf_df = annual_cf_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        annual_cf_df = annual_cf_df.add_suffix(f'(FY)')
    
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{tickerL}?period=quarterly&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    quarterly_cf_df = pd.DataFrame(response.json())
    if quarterly_cf_df.shape == (0, 0):
        quarterly_cf_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        quarterly_cf_df = quarterly_cf_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        quarterly_cf_df = quarterly_cf_df.add_suffix(f'(FQ)')
    
    stock_data = pd.concat([annual_is_df, quarterly_is_df, annual_fp_df, quarterly_fp_df, annual_cf_df  ], axis = 1)
    stock_data = pd.concat([stock_data,quarterly_cf_df], axis = 1)
    stock_data = stock_data[stock_data.columns.drop(list(stock_data.filter(regex='calendarYear')))]
    stock_data = stock_data[stock_data.columns.drop(list(stock_data.filter(regex='period')))]
    stock_data = stock_data.loc[:,~stock_data.columns.duplicated()].copy()
    stock_data1 = stock_data.iloc[0:9,]
    
    if quarterly_cf_df.shape[0]<9:
        for year in range(1,quarterly_cf_df.shape[0]):
            stock_data1 = pd.concat([stock_data1, pd.DataFrame(stock_data1.iloc[year,0:198].add_suffix(f'-{year}')).transpose().reset_index(drop=True)], axis=1)
    else:
        for year in range(1,9):
            stock_data1 = pd.concat([stock_data1, pd.DataFrame(stock_data1.iloc[year,0:198].add_suffix(f'-{year}')).transpose().reset_index(drop=True)], axis=1)
    stock_data1 = stock_data1.iloc[0,:]
    stock_data_final = pd.DataFrame(stock_data1).transpose()
    stock_data_final.insert(loc = 0, column ="Ticker", value=f"{tickerL}" )
    stock_data_final.insert(loc = 0, column ="Date(FY)", value=f"{last_date_annual}" )
    stock_data_final.insert(loc = 0, column ="Date(FQ)", value=f"{last_date_quarterly}" )
    
    return stock_data_final

In [30]:
us_stock_database = pd.DataFrame()
stock_list_pd = pd.read_pickle("StockList")
stock_list_pd = stock_list_pd.query("type == 'stock'")
stock_list_pd = stock_list_pd.query("exchangeShortName == 'NYSE' | exchangeShortName == 'NASDAQ'")
stock_list_pd.reset_index(inplace=True, drop=True)
stock_list = stock_list_pd["symbol"].to_list()
for ticker in tqdm(stock_list):
    #tqdm._instances.clear()
    us_stock_database = pd.concat([us_stock_database, get_stock_data(ticker)])
    #print(f'processed {ticker}')
    us_stock_database.reset_index(drop=True)


  0%|          | 28/10126 [03:33<21:21:04,  7.61s/it]


KeyboardInterrupt: 

In [31]:
us_stock_database

,Date(FQ),Date(FY),Ticker,revenue(FY),costOfRevenue(FY),grossProfit(FY),grossProfitRatio(FY),researchAndDevelopmentExpenses(FY),generalAndAdministrativeExpenses(FY),sellingAndMarketingExpenses(FY),...,netCashUsedProvidedByFinancingActivities(FQ)-8,effectOfForexChangesOnCash(FQ)-8,netChangeInCash(FQ)-8,cashAtEndOfPeriod(FQ)-8,cashAtBeginningOfPeriod(FQ)-8,operatingCashFlow(FQ)-8,capitalExpenditure(FQ)-8,freeCashFlow(FQ)-8,0,0-1
0,2023-12-31,2023-03-31,NGL-PB,8.694904e+09,7.923645e+09,7.712590e+08,0.088702,0.000000e+00,7.181800e+07,0.000000e+00,...,-1.345900e+07,0.0,-7.500000e+04,5.456000e+06,5.531000e+06,3.475600e+07,-2.967200e+07,5.084000e+06,NaN,NaN
0,2023-12-31,2023-12-31,NLY-PI,-1.651591e+09,2.480600e+07,-1.676397e+09,1.015019,0.000000e+00,4.296100e+07,0.000000e+00,...,8.792630e+08,0.0,2.957900e+08,1.342090e+09,1.046300e+09,8.615580e+08,-6.895900e+07,7.925990e+08,NaN,NaN
0,2023-12-31,2023-12-31,CIO-PA,1.790960e+08,6.999700e+07,1.090990e+08,0.609165,0.000000e+00,1.484100e+07,0.000000e+00,...,4.156100e+07,0.0,-2.791500e+07,4.226600e+07,7.018100e+07,7.491000e+06,0.000000e+00,7.491000e+06,NaN,NaN
0,2024-02-25,2023-11-26,LEVI,6.179000e+09,2.663300e+09,3.515700e+09,0.568976,0.000000e+00,1.267600e+09,1.803300e+09,...,-1.394290e+08,-443000.0,-1.322740e+08,6.783060e+08,8.105800e+08,8.613500e+07,-7.359100e+07,1.254400e+07,NaN,NaN
0,2023-12-31,2023-12-31,ICL,7.654434e+09,5.074516e+09,2.579918e+09,0.337049,7.211582e+07,4.360000e+08,9.170000e+08,...,-1.530000e+08,3000000.0,1.720000e+08,4.730000e+08,3.010000e+08,3.440000e+08,-1.850000e+08,1.590000e+08,NaN,NaN
0,2023-12-31,2023-12-31,CVX,2.009490e+11,1.666760e+11,3.427300e+10,0.170556,3.200000e+08,0.000000e+00,0.000000e+00,...,-8.290000e+09,-9000000.0,-2.300000e+08,6.795000e+09,7.025000e+09,9.458000e+09,-2.606000e+09,6.852000e+09,NaN,NaN
0,2023-12-31,2023-12-31,ALL-PH,5.709400e+10,7.137000e+09,4.995700e+10,0.874996,0.000000e+00,9.000000e+06,0.000000e+00,...,-1.175000e+09,-221000000.0,7.300000e+07,7.630000e+08,6.900000e+08,8.710000e+08,-5.900000e+07,8.120000e+08,NaN,NaN
0,2023-12-31,2023-12-31,BOC,9.625374e+07,8.424397e+07,1.200977e+07,0.124772,0.000000e+00,5.333969e+07,-3.256193e+07,...,1.246677e+07,0.0,-4.262000e+07,8.169440e+07,1.243144e+08,1.148140e+06,-9.540964e+06,-8.392824e+06,NaN,NaN
0,2023-12-31,2023-12-31,WTM,2.166700e+09,4.040000e+07,2.126300e+09,0.981354,0.000000e+00,4.326000e+08,0.000000e+00,...,-4.700000e+06,-400000.0,-5.140000e+07,2.593000e+08,3.107000e+08,-7.400000e+06,8.200000e+06,8.000000e+05,NaN,NaN
0,nan,nan,AEFC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
